# Exercises

Within your codeup-data-science directory, create a new directory named clustering-exercises. This will be where you do your work for this module. Create a repository on GitHub with the same name, and link your local repository to GitHub.

Save your clustering work in your clustering-exercises repo. Then add, commit, and push your changes.

For example, if the exercise directs you to create a file named myfile.py, you should have clustering/myfile.py in your repository.

If a file extension is specified, you should create that specific file. If there is not file extension specified, you may either create a python script or a jupyter notebook for the exercise.

In [1]:
# Imports
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import acquire
import summarize
import prepare


## Acquire & Summarize

1) Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
        
- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
        
- Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
        
- Only include properties that include a latitude and longitude value.

2) Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)
   
3) Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [2]:
df = acquire.get_zillow_data()

df.head()

,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,airconditioningtypeid,parcelid,id,architecturalstyletypeid,basementsqft,bathroomcnt,...,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate,airconditioningdesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc
0,nan,261.00,nan,None,nan,14297519,1727539,nan,nan,3.50,...,nan,"60,590,630,072,012.00",0,0.03,2017-01-01,None,None,None,Single Family Residential,None
1,nan,261.00,nan,None,nan,17052889,1387261,nan,nan,1.00,...,nan,"61,110,010,023,006.00",1,0.06,2017-01-01,None,None,None,Single Family Residential,None
2,nan,261.00,nan,None,nan,14186244,11677,nan,nan,2.00,...,nan,"60,590,218,022,012.00",2,0.01,2017-01-01,None,None,None,Single Family Residential,None
3,nan,261.00,2.00,None,nan,12177905,2288172,nan,nan,3.00,...,nan,"60,373,001,001,006.00",3,-0.10,2017-01-01,None,None,Central,Single Family Residential,None
4,nan,266.00,2.00,None,1.00,10887214,1970746,nan,nan,3.00,...,nan,"60,371,236,012,000.00",4,0.01,2017-01-01,Central,None,Central,Condominium,None


In [3]:
df = df.drop(columns = ['id'])

In [4]:
from summarize import df_summary
info, describe, nulls, value_count = df_summary (df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72240 entries, 0 to 72239
Data columns (total 65 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   storytypeid                   50 non-null     float64
 1   propertylandusetypeid         72240 non-null  float64
 2   heatingorsystemtypeid         46696 non-null  float64
 3   buildingclasstypeid           0 non-null      object 
 4   airconditioningtypeid         23079 non-null  float64
 5   parcelid                      72240 non-null  int64  
 6   architecturalstyletypeid      207 non-null    float64
 7   basementsqft                  50 non-null     float64
 8   bathroomcnt                   72240 non-null  float64
 9   bedroomcnt                    72240 non-null  float64
 10  buildingqualitytypeid         45079 non-null  float64
 11  calculatedbathnbr             71986 non-null  float64
 12  decktypeid                    612 non-null    float64
 13  f

## Prepare

1) Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

2) Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

### The input:
            
- A dataframe
            
- A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
            
- A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
        
### The output:
            
- The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
       
### hint:
            
- Look up the dropna documentation.
            
- You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
            
- Make use of inplace, i.e. inplace=True/False.

3) Decide how to handle the remaining missing values:
         - Fill with constant value.
         - Impute with mean, median, mode.
         - Drop row/column


### wrangle_zillow.py

Functions of the work above needed to acquire and prepare a new sample of data.
Mall Customers

In [5]:
from prepare import handle_missing_values

In [6]:
df = handle_missing_values(df, prop_required_column = .5, prop_required_row = .75)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72062 entries, 0 to 72239
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   propertylandusetypeid         72062 non-null  float64
 1   heatingorsystemtypeid         46696 non-null  float64
 2   parcelid                      72062 non-null  int64  
 3   bathroomcnt                   72062 non-null  float64
 4   bedroomcnt                    72062 non-null  float64
 5   buildingqualitytypeid         45078 non-null  float64
 6   calculatedbathnbr             71982 non-null  float64
 7   calculatedfinishedsquarefeet  72049 non-null  float64
 8   finishedsquarefeet12          71870 non-null  float64
 9   fips                          72062 non-null  float64
 10  fullbathcnt                   71982 non-null  float64
 11  latitude                      72062 non-null  float64
 12  longitude                     72062 non-null  float64
 13  l

# notebook

- Acquire data from mall_customers.customers in mysql database.
- Summarize data (include distributions and descriptive statistics).
- Detect outliers using IQR.
- Split data (train, validate, and test split).
- Encode categorical columns using a one hot encoder.
- Handles missing values.
- Scaling


# wrangle_mall.py

- Acquire data from mall_customers.customers in mysql database.
- Split the data into train, validate, and split
- One-hot-encoding
- Missing values
- Scaling